## Modelos lineales para clasificación

Podemos usar la función de la regresión lineal para intentar ajustar un modelo de clasificación:

<img src=https://amueller.github.io/ml-workshop-2-of-4/slides/images/linear_boundary_vector.png width=400>

$$\hat{y}_i = f(x_i) = \text{sign}\big(w_0 + w_1x_1 + w_2x_2 + \dots + w_dx_d\big)$$

Función de pérdida:

$$L(y_i, \hat{y_i}) = \sum_{i=1}^{n} \left\{ \begin{array}{cc} 0 & \text{si } y_i = \hat{y}_i \\ 1 & \text{si } y_i \neq \hat{y}_i \end{array} \right.$$

**Problema**: no podemos calcular derivadas, por tanto no podemos calcular los $w$ que minimizan la pérdida media

### Regresión logística

El equivalente a la regresión lineal para problemas de clasificación es la regresión logística. Reemplaza la pérdida zero-uno por una "aproximación", la pérdida logística:

$$L(y_i, \hat{y_i}) = \log(\exp(-y_i\hat{y}_i) + 1)$$

Comparación con la pérdida 0-1:

<img src=../../img/loss_comparison.png width=500>

In [1]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

blood = fetch_openml('blood-transfusion-service-center')

In [2]:
X_train, X_test, y_train, y_test = train_test_split(blood.data, blood.target, random_state=0)

In [3]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(penalty='none')
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

0.7165775401069518

In [4]:
import pandas as pd
prob = pd.DataFrame(lr.predict_proba(X_test), columns=['prob_clase_1', 'prob_clase_2'])
prob['clase'] = lr.predict(X_test)

In [5]:
def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]

prob.head(5).style.apply(highlight_max, subset=['prob_clase_1', 'prob_clase_2'], axis=1)

,prob_clase_1,prob_clase_2,clase
0,0.751464,0.248536,1
1,0.569721,0.430279,1
2,0.568277,0.431723,1
3,0.576968,0.423032,1
4,0.133358,0.866642,2


### Regresión logística regularizada

Al igual que añadimos un término de regularización a la regresión lineal, podemos añadirlo también a la regresión logística. En este caso no hay clases específicas, sino que hay que ajustar los valores del parámetro `penalty`:

   * `penalty=l1`, regularización $l_1$ (como Lasso)
   * `penalty=l2`, regularización $l_2$ (como Ridge)
   * `penalty=elasticnet`, regularización $l_1$ + regularización $l_2$ (como ElasticNet)
   
**Ojo**: por defecto la clase `LogisticRegression` incluye regularización $l_2$ (`penalty=l2`)

Podemos usar la clase `LogisticRegressionCV` para buscar automáticamente el valor óptimo de los parámetros `C` y `l1_ratio`

In [6]:
from sklearn.datasets import load_breast_cancer
breast = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(breast.data, breast.target, random_state=42)

In [7]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

/home/alberto/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.965034965034965

In [8]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

In [9]:
from sklearn.linear_model import LogisticRegressionCV

lrcv = LogisticRegressionCV(Cs=10, solver='liblinear')
lrcv.fit(X_train_std, y_train)
lrcv.score(X_test_std, y_test)

0.986013986013986

In [10]:
lrcv.C_

array([0.04641589])

### Métricas de clasificación

La mayoría de métricas se pueden derivar de la matriz de confusión:

<img src=../../img/confusion_matrix.png width=250>

 * Accuracy: $\frac{\text{TP} + \text{TN}}{\text{P} + \text{N}}$
 
 
 * Sensitivity, recall, TPR: $\frac{\text{TP}}{\text{TP} + \text{FN}}$


 * Specificity, TNR: $\frac{\text{TN}}{\text{TN} + \text{FP}}$


 * Precision, PPV: $\frac{\text{TP}}{\text{TP} + \text{FP}}$


 * F1 score: $2\times \frac{\text{PPV} \times \text{TPR}}{\text{PPV} + \text{TPR}}$

Las etiquetas positivo y negativo son arbitrarias, aunque se suelen denominar ejemplos positivos a la clase minoritaria.

 * Documentación API: [metrics](https://scikit-learn.org/stable/modules/classes.html#classification-metrics)
 
 * Guía de usuario: [Metrics and scoring: quantifying the quality of predictions](https://scikit-learn.org/stable/modules/model_evaluation.html#classification-metrics)

 * [Visualizations with Display Objects](https://scikit-learn.org/stable/auto_examples/miscellaneous/plot_display_object_visualization.html#sphx-glr-auto-examples-miscellaneous-plot-display-object-visualization-py)

In [11]:
from sklearn.metrics import confusion_matrix

ypred = lr.predict(X_test_std)
cm = confusion_matrix(y_test, ypred)

pd.DataFrame(cm, index=['real_0', 'real_1'], columns=['pred_0', 'pred_1'])

,pred_0,pred_1
real_0,26,28
real_1,42,47


In [12]:
tn, fp, fn, tp = cm.ravel()
print(tn)
print(tp)
print(fp)
print(fn)

26
47
28
42


### Problemas de clasificación no balanceados

[imbalanced-learn](https://imbalanced-learn.org/stable/)

In [5]:
import pandas as pd
df = pd.read_csv('../../data/balance-scale.data',  names=['balance', 'var1', 'var2', 'var3', 'var4'])

df['balance'] = [1 if b=='B' else 0 for b in df['balance']]
df['balance'].value_counts()

0    576
1     49
Name: balance, dtype: int64

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, f1_score, classification_report

X = df.drop(columns='balance').values
y = df['balance'].values

print(X.shape)
print(y.shape)

model = LogisticRegression(solver='liblinear')
model.fit(X, y)
ypred = model.predict(X)
print(model.score(X, y) * 100)

print(classification_report(y, ypred, zero_division=0))
print(confusion_matrix(y, ypred))

(625, 4)
(625,)
92.16
              precision    recall  f1-score   support

           0       0.92      1.00      0.96       576
           1       0.00      0.00      0.00        49

    accuracy                           0.92       625
   macro avg       0.46      0.50      0.48       625
weighted avg       0.85      0.92      0.88       625

[[576   0]
 [ 49   0]]


### Ejercicios

#### Ejercicio 1

Carga el conjunto de datos `adult.csv` usando pandas:

   * Transforma las variables categóricas usando codificación *one-hot*
   
   * Escala las variables para que tengan media 0 y desviación 1
   
   * Ajusta un modelo de regresión logística y visualiza los coeficientes
   
   * Busca el valor óptimo del parámetro $C$ usando validación cruzada
   
   * Prueba ahora con regularización `elasticnet` y busca el valor óptimo del parámetro `l1_ratio`
   
#### Ejercicio 2

Carga el conjunto de datos `titanic.csv` usando pandas:

   * Eliminar las variables `Cabin` y `Name`

   * Completar los valores que faltan en la edad con la media

   * Convertir las variables no numéricas en numéricas con `.get_dummies()`
   
   * Particionar el conjunto en train/test
   
   * Normalizar las variables para que tengan media 0 y varianza 1
   
   * Ajustar un modelo de Regresión Logística. Prueba con distintas regularizaciones
   
   * Realizar un gráfico de los coeficientes
   
   * Calcular la matriz de confusión, f1-score, precision y recall en el conjunto de test. ¿Como se interpretan estos resultados? ¿Observas algún problema?
   
   * Buscar los parámetros óptimos del modelo usando validación cruzada